In [1]:
from IPython.display import display

import numpy as np
import pandas as pd

This notebook calculates the ABD matrices from the (4) independent material properties required for orthotropic laminates; E11, E22, nu12, and G12.

In [2]:
E11 = 32.0e6
E22 = 20e6
G12 = 8.0e6
nu12 = 0.35
t_lam = 0.007
stack = [30]

In [3]:
nu21 = (nu12*E22)/E11
nu21

0.21875

In [4]:
Q11 = E11/(1-nu12*nu21)
Q11

34653130.28764805

In [5]:
Q22 = E22/(1-nu12*nu21)
Q22

21658206.429780032

In [6]:
Q12 = (nu12*E22)/(1-nu12*nu21)
Q12

7580372.250423011

In [7]:
Q66 = G12
Q66

8000000.0

In [8]:
t_total = len(stack)*t_lam
h = [(-1)*t_total/2]
h0 = h[0]
for i in range(len(stack)):
    hk = h0+t_lam
    h.append(hk)
    h0 = hk    
h_mat = pd.Series(h)
h_mat

0   -0.0035
1    0.0035
dtype: float64

In [9]:
m = []
for j in stack:
    M = np.cos((j*np.pi)/180)
    m.append(M)
m_mat = pd.Series(m)
m_mat

0    0.866025
dtype: float64

In [10]:
n = []
for l in stack:
    N = np.sin((l*np.pi)/180)
    n.append(N)
n_mat = pd.Series(n)
n_mat

0    0.5
dtype: float64

In [11]:
Qbar = np.zeros((len(stack),6))
for x in range(len(stack)):
    #Find Qbars:
    qbar11 = Q11*m[x]**4 + 2*(Q12+2*Q66)*m[x]**2*n[x]**2 + Q22*n[x]**4; Qbar[x][0] = qbar11
    qbar12 = (Q11+Q22-4*Q66)*m[x]**2*n[x]**2 + Q12*(m[x]**4 + n[x]**4); Qbar[x][1] = qbar12
    qbar16 = -m[x]*n[x]**3*Q22 + m[x]**3*n[x]*Q11 - m[x]*n[x]*(m[x]**2-n[x]**2)*(Q12+2*Q66); Qbar[x][2] = qbar16
    qbar22 = Q11*n[x]**4 + 2*(Q12+2*Q66)*m[x]**2*n[x]**2 + Q22*m[x]**4; Qbar[x][3] = qbar22
    qbar26 = -m[x]**3*n[x]*Q22 + n[x]**3*m[x]*Q11 + m[x]*n[x]*(m[x]**2-n[x]**2)*(Q12+2*Q66); Qbar[x][4] = qbar26
    qbar66 = (Q11+Q22-2*Q12)*m[x]**2*n[x]**2+Q66*(m[x]**2 - n[x]**2)**2; Qbar[x][5] = qbar66

names=['11', '12', '16', '22', '26', '66']
Qbar = pd.DataFrame(Qbar, columns=names)
Qbar

,11,12,16,22,26,66
0,2.968866e+07,9.296108e+06,3.804064e+06,2.319120e+07,1.822903e+06,9.715736e+06


In [12]:
A_mat = np.zeros((1,6))
for a in range(len(names)):
    thing = 0
    for i in range(len(stack)):
        thing += Qbar[names[a]][i]*(h_mat[i+1] - h_mat[i])
    A_mat[0][a] = thing
        
A_mat = pd.DataFrame(A_mat, columns=names)
A_mat

,11,12,16,22,26,66
0,207820.642978,65072.758037,26628.449471,162338.409475,12760.320163,68010.152284


In [13]:
B_mat = np.zeros((1,6))
for a in range(len(names)):
    thing = 0
    for i in range(len(stack)):
        thing += .5*(Qbar[names[a]][i]*(h_mat[i+1]**2 - h_mat[i]**2))
    B_mat[0][a] = thing
        
B_mat = pd.DataFrame(B_mat, columns=names)
B_mat

,11,12,16,22,26,66
0,0.0,0.0,0.0,0.0,0.0,0.0


In [14]:
D_mat = np.zeros((1,6))
for a in range(len(names)):
    thing = 0
    for i in range(len(stack)):
        thing += (Qbar[names[a]][i]*(h_mat[i+1]**3 - h_mat[i]**3))/3
    D_mat[0][a] = thing
        
D_mat = pd.DataFrame(D_mat, columns=names)
D_mat

,11,12,16,22,26,66
0,0.848601,0.265714,0.108733,0.662882,0.052105,0.277708


In [15]:
lams = []
for e in range(len(h_mat)-1):
    guy = h_mat.iloc[e+1]-h_mat.iloc[e]
    lams.append(guy)
Qbar['hk']=lams
Qbar['Ak']=Qbar['11']*Qbar['hk']
Qbar

,11,12,16,22,26,66,hk,Ak
0,2.968866e+07,9.296108e+06,3.804064e+06,2.319120e+07,1.822903e+06,9.715736e+06,0.007,207820.642978


In [16]:
A = np.array([[A_mat['11'], A_mat['12'], A_mat['16']], [ A_mat['12'], A_mat['22'], A_mat['26']], [A_mat['16'], A_mat['26'], A_mat['66']]])
A

array([[[ 207820.642978  ],
        [  65072.75803723],
        [  26628.44947136]],

       [[  65072.75803723],
        [ 162338.40947547],
        [  12760.3201627 ]],

       [[  26628.44947136],
        [  12760.3201627 ],
        [  68010.15228426]]])

In [17]:
B = np.array([[B_mat['11'], B_mat['12'], B_mat['16']], [ B_mat['12'], B_mat['22'], B_mat['26']], [B_mat['16'], B_mat['26'], B_mat['66']]])
B

array([[[ 0.],
        [ 0.],
        [ 0.]],

       [[ 0.],
        [ 0.],
        [ 0.]],

       [[ 0.],
        [ 0.],
        [ 0.]]])

In [18]:
D = np.array([[D_mat['11'], D_mat['12'], D_mat['16']], [ D_mat['12'], D_mat['22'], D_mat['26']], [D_mat['16'], D_mat['26'], D_mat['66']]])
D

array([[[ 0.84860096],
        [ 0.26571376],
        [ 0.10873284]],

       [[ 0.26571376],
        [ 0.66288184],
        [ 0.05210464]],

       [[ 0.10873284],
        [ 0.05210464],
        [ 0.27770812]]])